In [ ]:
import pandas as pd


df=pd.read_parquet('C:/Users/91847/Downloads/train', engine="pyarrow")


In [ ]:
df.head(5)

In [ ]:
df.to_csv('etsy_train.csv')

In [ ]:
f"Number of rows in train: {len(df):,}"

In [ ]:
f"Number of unique products in train: {len(df['product_id'].unique()):,}"

In [ ]:
f"Number of columns in train: {len(df.columns):,}"

In [ ]:
f"Number of unique top category in train: {len(df['top_category_id'].unique()):,}"

In [ ]:
f"Number of unique bottom category in train: {len(df['bottom_category_id'].unique()):,}"

In [ ]:
f"Number of colour category in train: {len(df['color_id'].unique()):,}"

## DEALING WITH NULL VALUES

In [ ]:
#Checking NULL from the title column
null_values = df['title'].isnull().sum()
print('Number of null values in the Age column:', null_values)

In [ ]:
#Removing NULL from the title
df = df.dropna(subset=['title'])

In [ ]:
#Checking NULL from the descriotion column
null_values = df['description'].isnull().sum()
print('Number of null values in the Description column:', null_values)

In [ ]:
#Checking NULL from the descriotion column
null_values = df['tags'].isnull().sum()
print('Number of null values in the Tags column:', null_values)

In [ ]:
#Removing NULL from the tags
df = df.dropna(subset=['tags'])

In [ ]:
#Checking top_category_text/id from the descriotion column
null_values = df[[0]].isnull().sum()
print('Number of null values in the Tags column:', null_values)

In [ ]:
df[['color_id']] = df[['color_id']].apply(pd.to_numeric, errors='coerce')

In [ ]:
#Saving Cleaned Data

# df.to_csv('etsy_train_clean.csv')

In [ ]:
unique_bci = df['top_category_id'].unique()
print (len(unique_bci))
type(unique_bci)

In [ ]:
#Selecting the important columns only to predict top category id
df2_top = df.iloc[: , [1, 2, 3, 17]].copy()
df2_top.head(5)

In [ ]:
import pandas as pd
#Selecting the important columns only to predict bottom category id
df2_bottom=pd.read_csv('C:/Users/91847/Desktop/DCU/Assignments/Etsy ML/etsy_train_clean1.csv')
df2_bottom = df2_bottom.iloc[: , [1, 2, 3, 4, 18, 16]].copy()
df2_bottom.head(10)

In [ ]:
#Merging the title description and tags into a single column
#Top Category Prediction
df2_top['text'] = df2_top['title'] + ' ' + df2_top['description'] + ' ' + df2_top['tags']

In [ ]:
#Merging the title description and tags into a single column
#Bottom Category Prediction
df2_bottom['text'] = df2_bottom['title'] + ' ' + df2_bottom['description'] + ' ' + df2_bottom['tags']

In [ ]:
df2_bottom.head(3)

In [ ]:
#Re-arranging the columns 
df3_top=df2_top[['text', 'top_category_id']]

In [ ]:
#Re-arranging the columns 
df3_bottom=df2_bottom[['product_id','text', 'top_category_id', 'bottom_category_id']]

In [ ]:
df3_bottom.head(2)

In [ ]:
import pandas as pd
#Reading the data into CSV for Top category Prediction
df3_top=pd.read_csv('C:/Users/91847/Desktop/DCU/Assignments/Etsy ML/etsy_train_top_catagory_prediction1.csv')

In [ ]:
#Saving the data into CSV for Bottom category Prediction
df3_bottom.to_csv('C:/Users/91847/Desktop/DCU/Assignments/Etsy ML/etsy_train_bottom_catagory_prediction3.csv')

In [ ]:
import pandas as pd

df3_bottom=pd.read_csv('etsy_train_bottom_catagory_prediction3.csv')

## Cleaning the text

In [ ]:
df3_bottom.shape[0]
#df3_bottom.head(10)

#df3_top.shape[0]

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [ ]:
#For Top Category

# corpus = []
# for i in range(0, 210564):  
#   text = re.sub('[^a-zA-Z]', ' ', df3_top['text'][i])
#   text = text.lower()
#   text = text.split()
#   ps = PorterStemmer()
#   all_stopwords = stopwords.words('english')
#   all_stopwords.remove('not')
#   text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
#   text = ' '.join(text)
#   corpus.append(text)

#For Bottom Category

corpus = []
for i in range(0, 210570):  
  text = re.sub('[^a-zA-Z]', ' ', df3_bottom['text'][i])
  text = text.lower()
  text = text.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
  text = ' '.join(text)
  corpus.append(text)

## Creating the Bag of Words model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 600) #max_features = 'n' to reduce the feature matrix
X = cv.fit_transform(corpus).toarray()
#y = df3_top.iloc[:, 2].values
y = df3_bottom.iloc[:, 4].values

In [ ]:
z=df3_bottom.iloc[:, 3].values
product_id=df3_bottom.iloc[:, 1].values

In [ ]:
product_id

In [ ]:
# Adding topid to feature matrix
import numpy as np
X = np.column_stack((X,z))


In [ ]:
X

## Using OverSampling (SMOTE)

In [ ]:
from imblearn.over_sampling import SMOTE
X_over, y_over = SMOTE().fit_resample(X,y)

In [ ]:
#checking balance of data
from collections import Counter
Counter(y_over)

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size = 0.20)

## Training the Random Forest model on the Training set

In [ ]:
# #FOR Naive Bayes (bas_model)
# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# classifier.fit(X_train, y_train)

#Random Forest Classification (Top_category_prediction)
# from sklearn.ensemble import RandomForestClassifier
# classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy')
# classifier.fit(X_train, y_train)

#K-Nearest Neighbour (Bottom_Category_Prediction)
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 500, weights='distance', metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)


In [ ]:
#importing test data

In [ ]:
import numpy as np
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
#cm = confusion_matrix(y_test, y_pred)
#print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred,average='weighted')

In [ ]:
precision_score(y_test, y_pred, average='weighted')

In [ ]:
recall_score(y_test, y_pred, average='weighted')

## Colour Prediction(Performed separately at kaagle notebook)

In [ ]:
%pip install tensorflow tensorflow-gpu opencv-python matplotlib
%pip list

In [ ]:
import tensorflow as tf
import os

In [ ]:
# GPU Memory Consumption Growth to block out of memory error
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
#Checking the GPU detection
tf.config.list_physical_devices('GPU')

In [ ]:
#Removing images from different extensions
data_dir = 'data'
image_exts = ['jpeg','jpg', 'bmp', 'png']
data_dir="/kaggle/input/etsy-colour/Etsy/data/train"
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

data = tf.keras.utils.image_dataset_from_directory('/kaggle/input/etsy-colour/Etsy/data/train')
data_iterator = data.as_numpy_iterator()
data_iterator = data.as_numpy_iterator()

#checking 6 images
fig, ax = plt.subplots(ncols=4, figsize=(15,15))
for idx, img in enumerate(batch[0][:6]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])    

In [ ]:
# Scaling the data
data = data.map(lambda x,y: (x/255, y))
data.as_numpy_iterator().next()

In [ ]:
#Splitting the data into train-test & validation
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)+1

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
#Building the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
# Define the ResNet50 model
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

# Freeze all the layers
for layer in base_model.layers:
    layer.trainable = False

from tensorflow.keras import layers
from keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D,AveragePooling2D
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(6, activation='softmax')(x)


from tensorflow.keras.models import Model
model=Model(inputs=base_model.input,outputs=predictions)

In [ ]:
#checking Summary of the model
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

#hist = model.fit(train_generator, epochs=20, validation_data=val, callbacks=[tensorboard_callback])
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
#Plotting the performance 
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()